In [18]:
%pip install huggingface-hub safetensors ../

Processing /Users/jakub/Documents/code/HIPPO
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hippo: filename=hippo-0.1.dev1+g5ed50b5.d20240901-py3-none-any.whl size=17316 sha256=fd2fb6409597d060fa75282ed022cce949b3b8c6aa26b87c9d6ddc712e6ebfff
  Stored in directory: /private/var/folders/29/_vp3243s5t1f3lvrkydsjchw0000gn/T/pip-ephem-wheel-cache-cu0xfh25/wheels/c4/71/f0/0684c0397b0938e3d3bc8fa8c26240978f0c2a987b1bb6647e
Successfully built hippo
  Attempting uninstall: hippo
    Found existing installation: hippo 0.1.dev1+g5ed50b5.d20240901
    Uninstalling hippo-0.1.dev1+g5ed50b5.d20240901:
      Successfully uninstalled hippo-0.1.dev1+g5ed50b5.d20240901
Note: you may need to restart the kernel to use updated packages.


In [2]:
s = """
python scripts/train_classification.py \
--model-name AttentionMILModel \
--features-dir /home/jkaczmar/data/public-wsi/deep_features/{encoder}-256px_0.5mpp/camelyon16/ \
--output-dir outputs/camelyon16/abmil-{encoder}-128um_seed{seed} \
--csv csvs/camelyon16-jakub-labels.csv \
--label-col binary_label_int \
--num-classes 2 \
--embedding-size {embedding_size} \
--split-json splits/camelyon16-split0.json \
--fold 0 \
--num-epochs 20 \
--seed {seed} \
-L 512 \
-D 384 \
--lr 1e-4"""

encoders_sizes = [
    ("UNI", 1024),
#     ("CTransPath", 768),
#     ("REMEDIS", 4096),
#     ("RetCCL", 2048),
#     ("Phikon", 768),
]

for encoder, embedding_size in encoders_sizes:
    for seed in [0]:
        print(s.format(encoder=encoder.lower(), embedding_size=embedding_size, seed=seed).strip())

python scripts/train_classification.py --model-name AttentionMILModel --features-dir /home/jkaczmar/data/public-wsi/deep_features/uni-256px_0.5mpp/camelyon16/ --output-dir outputs/camelyon16/abmil-uni-128um_seed0 --csv csvs/camelyon16-jakub-labels.csv --label-col binary_label_int --num-classes 2 --embedding-size 1024 --split-json splits/camelyon16-split0.json --fold 0 --num-epochs 20 --seed 0 -L 512 -D 384 --lr 1e-4


In [21]:
from hippo.models import AttentionMILModel
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

weights_path = hf_hub_download(
    repo_id="kaczmarj/breast-lymph-nodes-metastasis.camelyon16",
    filename="model.safetensors"
)
state_dict = load_file(weights_path)
model = AttentionMILModel(in_features=768, L=256, D=256, num_classes=2, gated_attention=True)
model.eval()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# Load patch features, made with UNI model.
torch.load("")

### Get the model outputs on the original image

<img src="../docs/cam16-test_002-original.jpg" width="384px">

### Get the model outputs on the without tumor

<img src="../docs/cam16-test_002-withouttumor.jpg" width="384px">

In [23]:
tumor_polygons = ...

### Remove tumor and get model outputs

<img src="../docs/cam16-test_002-tumor.jpg" width="384px">